# **Identification de 95 langues avec des :**
>### **- '*Sparse*' Bag Of Words**
>### **- Tokenisations BERT ou Tiktoken**
>### **- CountVectorizer utilisant une tokenisation '*custom*'**
>### **- Classificateurs Naïve Bayes et Gradiant Boosting**

## **1 - Contruction des classificateurs**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import joblib
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Choix de la Tokenisation (False = BERT, True Tiktoken)
titoken_tokenization = True

# Ce parametre permet éventuellement d'équilibrer de nombre de phrase par langue.
# Si ce parametre est très grand, tout le corpus sera lu. 
nb_phrase_lang = 10000000

import warnings
warnings.filterwarnings('ignore')

#### **Lectures des phrases de "sentences.csv", et de leur étiquette "Langue" pour les langues sélectionnées**

In [2]:
# Ouvrir le fichier d'entrée en mode lecture
def create_lang_df(path):
    df = pd.read_csv(path, index_col ='id')
    return df

df = create_lang_df('../data/multilingue/sentences-big.csv')
lan_code = list(set(df['lan_code']))
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
n_rows = len(df)
print('Nombre de lignes de sentence.csv:',n_rows)
print('Nombre de langues à classer:',len(lan_code))
df

Nombre de lignes de sentence.csv: 10341812
Nombre de langues à classer: 404


,lan_code,sentence
0,ita,Il tuo futuro è pieno di possibilità.
1,fra,"J'aimerais aller en France, un jour."
2,epo,La polica enketo aperigis ilian sekretan vivon.
3,kab,Kullec ifukk yid-k.
4,hun,Több munkát nem tudok elvállalni.
...,...,...
10341807,deu,Wir werden das Problem nicht aufgreifen.
10341808,fra,Je suis cuit !
10341809,kab,Isefk fell-ak a tregleḍ iɣis-a.
10341810,tok,o pana ala e moku tawa soweli tomo.


#### **Réalisation d'un jeu de données d'entrainement et de test**

In [25]:
# créer 2 dataframes: 1 train (95% des phrases) et 1 test (5% des phrases)
n_train = int(n_rows*0.95)
df_train = df.iloc[:n_train].sample(frac=1, random_state=42).reset_index(drop=True)
df_test = df.iloc[n_train:].sample(frac=1, random_state=24).reset_index(drop=True)
pd.set_option('display.max_colwidth', 150)
df_lan = pd.DataFrame(data= df.groupby('lan_code').size(), columns = ['nb_phrases_lang'] )

# Filtrage des langues qui ont peu de phrases (>2000)
df_lan = df_lan.loc[df_lan['nb_phrases_lang']>=2000]
list_lan = list(set(df_lan.index))
df_train = df_train[df_train['lan_code'].isin(list_lan)]
df_test = df_test[df_test['lan_code'].isin(list_lan)]
print('df_train:')
display(df_train)
print('Nombre de lignes par langue:')
display(df_lan)


df_train:


,lan_code,sentence
0,cmn,咱们停止争吵和好吧。
1,fra,Sami a acheté une bouteille de vin.
2,fra,Tom a du mal à comprendre ce concept.
3,rus,Они идут друг за другом.
4,spa,Quisiera enviar este paquete a Japón.
...,...,...
9824715,rus,Давайте смотреть фактам в лицо!
9824717,jpn,私はそれについて全く知りません。
9824718,rus,Как я сегодня счастлив!
9824719,epo,"Mi esperas, ke baldaŭ ni povu kunlabori."


Nombre de lignes par langue:


,nb_phrases_lang
lan_code,
afr,4137
ara,38650
arq,2326
asm,3205
avk,4102
...,...
war,2025
wuu,4757
yid,9603


#### **Selection du tokenizer** en fonction de la variable titoken_tokenization

In [26]:
# Selection du tokenizer
if titoken_tokenization:
    import tiktoken
    tokenizer = tiktoken.get_encoding("cl100k_base")
else:
    from transformers import BertTokenizerFast
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')

#### **Préparation de la vectorisation par CountVectorizer**

In [27]:
# Les 2 fonctions suivantes sont nécéssaires afin de sérialiser ces parametre de CountVectorizer
# et ainsi de sauvegarder le vectorizer pour un un usage ultérieur sans utiliser X_train pour  le réinitialiser
def custom_tokenizer(text):
    tokens = tokenizer.encode(text)  # Cela divise le texte en mots
    return tokens

def custom_preprocessor(text):
    return text

# CountVectorizer a une liste de phrase en entrée.
# Cette fonction met les données d'entrée dans le bon format
def format_to_vectorize(data):
    X_tok = []
    if "DataFrame" in str(type(data)):sentences = df.tolist()
    elif "str" in str(type(data)):
        sentences =[data]
    else: sentences = data
                          
    for sentence in sentences:
        X_tok.append(sentence) # ('¤'.join([tokenizer.decode([ids]) for ids in tokenizer.encode(sentence)])+'¤')
    return X_tok

#### **Création de la fonction Vectorizer et de la fonction de création d'un Bags  Of Worlds**

In [28]:
# Création d'un vectorizer et du sparse BOW (X_train) avec le nombre d'apparitions
global vectorizer, dict_ids, dict_token

def create_vectorizer(X_train_tok):
    global vectorizer, dict_ids, dict_token
    
    # token_pattern = r"[a-zA-Z0-9\s\.\,\?\:\;]+" 
    # vectorizer = CountVectorizer(analyzer="word", tokenizer=lambda x: tokenizer.encode(x), preprocessor=lambda x: x) #,token_pattern=token_pattern
    vectorizer = CountVectorizer(analyzer="word", tokenizer=custom_tokenizer, preprocessor=custom_preprocessor) #,token_pattern=token_pattern
    vectorizer.fit(X_train_tok)
    
    # Création de dictionnaire des Token et des ids 
    dict_token = {tokenizer.decode([cle]): cle for cle, valeur in vectorizer.vocabulary_.items()}
    dict_ids = {cle: tokenizer.decode([cle]) for cle, valeur in vectorizer.vocabulary_.items()} #dict_ids.items()}
    return 

def create_BOW(data, vectorizer_to_create=False):
    global vectorizer
    
    X_tok = format_to_vectorize(data)
    if vectorizer_to_create:
        create_vectorizer(X_tok)
    X = vectorizer.transform(X_tok)
    return X

#### **Création du BOW Train**

In [37]:
X_train = create_BOW(df_train['sentence'], True)
y_train = df_train['lan_code'].values.tolist()

#### **Création du BOW Test**


In [38]:
X_test = create_BOW(df_test['sentence'])
y_test = df_test['lan_code'].values.tolist()

#### **Sauvegarde/Chargement du vectorizer**

In [39]:
# Definition de fonction de sauvegarde et chargement du dictionnaire des tokens utilisés
def save_vectorizer(vectorizer):
    if titoken_tokenization: path = '../data/vectorizer_tiktoken_big.pkl'
    else: path = '../data/vectorizer_BERT.pkl' 
    joblib.dump(vectorizer, path)

def load_vectorizer():
    global dict_token, dict_ids, nb_token
    
    if titoken_tokenization: path = '../data/vectorizer_tiktoken_big.pkl'
    else: path = '../data/vectorizer_BERT.pkl'
    vectorizer = joblib.load(path)
    dict_token = {tokenizer.decode([cle]): cle for cle, valeur in vectorizer.vocabulary_.items()}
    dict_ids = {cle: tokenizer.decode([cle]) for cle, valeur in vectorizer.vocabulary_.items()} #dict_ids.items()}
    nb_token = len(vectorizer.vocabulary_)
    return vectorizer

In [40]:
save_vectorizer(vectorizer)

vectorizer = load_vectorizer()

#### **Definition d'une fonction ids->colonne de X_train et col->ids**

In [41]:
def ids2col(list_ids):
    d = dict(vectorizer.vocabulary_.items())
    list_col = []
    for ids in list_ids:
        list_col.append(d[ids])
    return list_col

def col2ids(list_col):
    d = dict(vectorizer.vocabulary_.items())
    list_ids = []
    for col in list_col:
        for ids, c in d.items():
            if col==c:
                list_ids.append(ids)
                break
    return list_ids

#### **Création d'un dictionnaire des tokens avec leur fréquence d'apparition dans Train**
#### **Définition d'une liste de token trié par fréquence d'apparition**

In [56]:
freq = X_train.sum(axis=0)
list_ids = col2ids(range(len(dict_ids)))
list_token = [tokenizer.decode([ids]) for ids in list_ids]
dict_freq = dict(zip(list_token,freq.tolist()[0]))
dict_freq = dict(sorted(dict_freq.items(), key=lambda x: x[1], reverse=True))

def ids2token(ids):
    for token, valeur in dict_ids.items():
        if valeur == ids:
            token_trouvee = token
            break
    return token_trouvee

nb_token = len(vectorizer.vocabulary_)
print("Nombre de tokens :",nb_token)

# Définition d'une liste 'écrite' des tokens : decoded_keys
# decoded_keys = [tokenizer.decode([ids]) for ids in [ids2token(key) for key in list(dict_freq.keys())]]
decoded_keys = list(dict_freq.keys())
print("Liste des 50 tokens les plus fréquents:",decoded_keys[:50])

# vocab_size = max(max(row) for row in X_train) + 1

Nombre de tokens : 59122
Liste des 50 tokens les plus fréquents: ['.', ',', '?', ' a', 'i', 'Tom', 'a', 'u', ' to', ' la', ' de', ' ', ' t', ' d', 'I', 'as', ' Tom', ' k', 'е', 'en', ' n', 'а', 'и', ' in', ' the', ' y', 'ו�', 'is', 'у', 'o', 'о', 'ом', ' с', 'er', ' в', 't', 'z', 'T', '!', ' that', 'ı', '。', ' i', "'t", 'י�', ' ad', 'in', ' ne', 'em', ' is']


#### **Choix du nom du fichier de sauvegarde du classifieur**

In [42]:
def get_file_name(titoken_tokenization, classifier):
    if titoken_tokenization:
        return "id_lang_tiktoken_"+classifier+"_sparse-big.pkl"
    else:
        return "id_lang_BERT_"+classifier+"_sparse-big.pkl"

#### **Création d'un classificateur avec l'algorithme Naïve Bayes**

In [44]:
from sklearn import naive_bayes

# On definit le classificateur Naive Bayes et on l'entraine sur les données Train:
clf_nb = naive_bayes.MultinomialNB()  # BernoulliNB() # MultinomialNB() 
clf_nb.fit(X_train, y_train)

# Save the model to a file
joblib.dump(clf_nb, "../data/"+get_file_name(titoken_tokenization,"nb")) ######### 

['../data/id_lang_tiktoken_nb_sparse-big.pkl']

In [45]:
# Chargement du classificateur sauvegardé
clf_nb = joblib.load("../data/"+get_file_name(titoken_tokenization,"nb"))

# Verification de l'efficacité du classificateur grace à la matrice confusion
y_pred = clf_nb.predict(X_test)
accuracy_naive_bayes = accuracy_score(y_test, y_pred)
print("Matrice de confusion du classificateur Naïve Bayes")
ct = pd.crosstab(y_test,y_pred,rownames=['Classe réelle'], colnames=['Classe prédite'])
display(ct)
print("Accuracy Naïve Bayes = {:.3f}".format(accuracy_naive_bayes))

Matrice de confusion du classificateur Naïve Bayes


Classe prédite,afr,ara,arq,asm,avk,aze,bel,ben,ber,bre,...,uig,ukr,urd,vie,vol,war,wuu,yid,yue,zsm
Classe réelle,,,,,,,,,,,,,,,,,,,,,
afr,147,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ara,0,1901,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
arq,0,99,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
asm,0,0,0,106,0,0,0,69,0,0,...,0,0,0,0,0,0,0,0,0,0
avk,0,0,0,0,174,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
war,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,73,0,0,0,0
wuu,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,75,0,2,0
yid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,430,0,0


Accuracy Naïve Bayes = 0.960


In [49]:
from sklearn import naive_bayes
ct_weurope = ct[['eng','deu','fra','ita','spa']].loc[['eng','deu','fra','ita','spa']]
s = ct_weurope.sum().sum()
acc = (ct_weurope.loc['eng','eng']+ct_weurope.loc['deu','deu']+ct_weurope.loc['fra','fra']+ct_weurope.loc['ita','ita']+ct_weurope.loc['spa','spa'])/s
print("Accuracy Naïve Bayes sur eng, deu, fran ita, spa = {:.3f}".format(acc))

Accuracy Naïve Bayes sur eng, deu, fran ita, spa = 0.999


#### **Definition de fonction identificateur de langue**

In [50]:
def lang_id_nb(sentences):
    if "str" in str(type(sentences)):
        return clf_nb.predict(create_BOW(sentences))[0]
    else: return clf_nb.predict(create_BOW(sentences))
   

#### **Exemples d'utilisation**

In [51]:
# Instanciation d'un exemple
exemples = ["Er weiß überhaupt nichts über dieses Buch.",                                                             # Phrase 0
            "france is often snowy during spring , and it is relaxing in january .",                                  # Phrase 1
           "elle adore les voitures très luxueuses, et toi ?",                                                        # Phrase 2
           "she loves very luxurious cars, don't you?",                                                               # Phrase 3
           "vamos a la playa",                                                                                        # Phrase 4
           "Ich heiße Keyne, und das ist wunderbar",                                                                  # Phrase 5
           "she loves you much, mais elle te hait aussi and das ist traurig.", # Attention à cette phrase trilingue   # Phrase 6
           "A crane raises heavy construction materials.",                                                            # Phrase 7
           "Vogliamo visitare il Colosseo e nuotare nel Tevere."                                                      # Phrase 8
          ]
lang_exemples = ['deu','eng','fra','eng','spa','deu','en,fr,de','en','ita']

In [52]:
# Affichage des prédictions
print('Langue réelle                 :',lang_exemples)
print('Prédictions Naive Bayes       :',lang_id_nb(exemples))


Langue réelle                 : ['deu', 'eng', 'fra', 'eng', 'spa', 'deu', 'en,fr,de', 'en', 'ita']
Prédictions Naive Bayes       : ['deu' 'eng' 'fra' 'eng' 'spa' 'deu' 'glg' 'eng' 'ita']


> **Recherche des phrases mal classées par Naive Bayes**

In [55]:
n_bad_max = 30
n_bad = 0
for i in range(len(y_test)):
    if (y_test[i] != y_pred[i]) and (n_bad<n_bad_max):
        n_bad +=1
        print(n_bad,'- No',i,' - Réel:',y_test[i],' Prédit:',y_pred[i],'    ',df_test['sentence'].iloc[i]," (proba={:.2f}".format(max(clf_nb.predict_proba(create_BOW([df_test['sentence'].iloc[i]]))[0]))+")")


1 - No 92  - Réel: mkd  Prédit: rus      Не сум ни почнал.  (proba=0.78)
2 - No 249  - Réel: eng  Prédit: gos      Haijo loves frikandels.  (proba=0.99)
3 - No 297  - Réel: kab  Prédit: ber      Ameskar-a d win yessenqaden imidyaten imensayen.  (proba=1.00)
4 - No 307  - Réel: gos  Prédit: nld      Doe hest dien buusdouk valen loaten.  (proba=0.88)
5 - No 314  - Réel: grn  Prédit: tur      Oreroryete.  (proba=0.46)
6 - No 339  - Réel: cmn  Prédit: kmr      法語包含二十六個字母: a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z。  (proba=1.00)
7 - No 352  - Réel: ber  Prédit: kab      Tom yenǧeε Mary deg uεrur.  (proba=0.67)
8 - No 354  - Réel: ile  Prédit: epo      Ili parla pri te, Mary.  (proba=0.94)
9 - No 425  - Réel: kab  Prédit: ber      Ur lliɣ ara d tayemmat yelhan.  (proba=0.78)
10 - No 438  - Réel: ukr  Prédit: rus      Том теж спить.  (proba=0.91)
11 - No 465  - Réel: srp  Prédit: mkd      Она ће данас поподне да опере бицикл.  (proba=0.77)
12 - No 501  - Rée